# Agentic RAG Playground by LlamaIndex

In [1]:
import datasets
from llama_index.core.schema import Document

In [2]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [3]:
# Initialize the tool
from src.tools import GuestInfoRetrieverTool
guest_info_tool = GuestInfoRetrieverTool(docs)

In [6]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.tools import FunctionTool
from tools import GuestInfoRetrieverTool

# Initialize the tool
guest_info_retriever = GuestInfoRetrieverTool(docs)

# Create a proper FunctionTool from the method
guest_info_tool = FunctionTool.from_defaults(
    fn=guest_info_retriever.get_guest_info,
    name="guest_info",
    description="Retrieve information about guests attending the gala by their name or details"
)

# Initialize the Hugging Face model
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")

# Create Alfred, our gala agent, with the guest info tool
alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool],  # Now this is a proper FunctionTool
    llm=llm,
)

# Example query Alfred might receive during the gala
response = await alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)


WorkflowRuntimeError: Error in step 'run_agent_step': 401, message='Unauthorized', url='https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions'